Crear una red convolucional con el dataset de tensorflow
Análisis exploratorio de los datps
Guardar el modelo con .save
Hacer Predicciones con imagenes
Realizar metricas de evaluación

instalar Librerias

In [2]:
!pip install tensorflow-datasets

In [8]:
!pip install tensorflow-datasets fastai

In [1]:
!pip install fastai==2.7.12
!pip install tensorflow-datasets==4.9.2

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52

In [3]:
from fastai.vision.all import *
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from fastai.vision.all import *


Cargar el Dataset

In [4]:
ds, info = tfds.load('imagenette', with_info=True, as_supervised=True)

El objeto info contiene información sobre el dataset, incluyendo las etiquetas. Podemos acceder a ellas de la siguiente manera

In [5]:
nombres_etiquetas = info.features['label'].names

Necesitamos convertir el dataset de TFDS a un objeto DataLoaders de fastai para el entrenamiento. Primero, creamos una función para decodificar las imágenes (ya que están almacenadas como JPEG en el dataset)

In [11]:
def decodificar_imagen(imagen, etiqueta):
  imagen = tf.image.convert_image_dtype(imagen, dtype=tf.float32)
  return imagen, etiqueta

Ahora, aplicamos esta función al dataset y creamos el DataLoaders:

In [12]:
!pip install tensorflow tensorflow-datasets fastai

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
from fastai.vision.all import *

# Define a function to get the image and label from a TensorFlow Dataset element
def get_x(x): return x['image']
def get_y(x): return x['label']

In [15]:
# Define the blocks for the DataBlock
blocks = (ImageBlock, CategoryBlock)



In [16]:
# **Change 1:** Concatenate train and validation datasets into a single list
ds_list = list(ds['train']) + list(ds['validation'])

NameError: name 'ds' is not defined

In [5]:
# **Change 2:** Modify the splitter to use a RandomSplitter
# We can't use GrandparentSplitter directly with a list of items.
splitter = RandomSplitter(valid_pct=0.2) # 20% for validation

# Create the DataBlock
dblock = DataBlock(blocks=blocks,
                   get_x=get_x,
                   get_y=get_y,
                   splitter=splitter,
                   item_tfms=Resize(224), # Resize images
                   batch_tfms=aug_transforms()) # Data augmentation

In [6]:
# Load the data using the DataBlock, passing the combined list
dls = dblock.dataloaders(ds_list) # Pass the combined list of items

NameError: name 'ds_list' is not defined

Podemos usar un modelo pre-entrenado como ResNet18 y ajustarlo para nuestra tarea.

In [2]:
from fastai.vision.all import *
from fastai.callback.all import *

# ... (código anterior para crear dls) ...

# Crea un "learner" de visión
learn = vision_learner(dls, resnet18, metrics=accuracy)

# Encuentra una buena tasa de aprendizaje
learn.lr_find()

# Ajusta el modelo con callbacks
learn.fine_tune(epochs=5,
                cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3),
                     SaveModelCallback(monitor='valid_loss')])

NameError: name 'dls' is not defined

Entrenar Modelo

In [ ]:
learn.fine_tune(5)

Pruebas con Imágenes Aleatorias y Evaluación
Cargar Imágenes Aleatorias
Para probar con imágenes aleatorias, puedes descargar algunas imágenes de internet y guardarlas en una carpeta, por ejemplo, 'imagenes_prueba'. Luego, cárgalas en un DataLoader

In [ ]:
test_dl = learn.dls.test_dl(get_image_files('imagenes_prueba'))

Haciendo predicciones

In [ ]:
preds, targets = learn.get_preds(dl=test_dl)

Metricas de Evaluacion:

In [ ]:
precision = accuracy_multi(preds, targets)
print(f'Precisión: {precision}')